In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [83]:
kcbert_pred = np.load('/aiffel/aiffel/aiffelthon/submission/kcBERT_ep1.npy', allow_pickle=True)

# labels = {'갈취 대화': '01', '기타 괴롭힘 대화': '03', '일반 대화': '04', '직장 내 괴롭힘 대화': '02', '협박 대화': '00'}
kcbert_pred_2 = []

for pre in kcbert_pred:
    a = []
    a.append(pre[4]['score'])
    a.append(pre[0]['score'])
    a.append(pre[3]['score'])
    a.append(pre[1]['score'])
    a.append(pre[2]['score'])
    kcbert_pred_2.append(a)
    
kcbert_pred = np.array(kcbert_pred_2)   

In [10]:
kcelectra_pred = np.load('/aiffel/aiffel/aiffelthon/submission/kcELECTRA.npy', allow_pickle=True)
kcelectra_pred[0]

array([4.0593333e-04, 9.9872029e-01, 1.5850972e-04, 6.2708918e-04,
       8.8240442e-05], dtype=float32)

In [70]:
kluebert_pred = np.load('/aiffel/aiffel/aiffelthon/submission/klue_bert.npy', allow_pickle=True)

kluebert_pred_2 = []

for pre in kluebert_pred:
    a = []
    a.append(pre[4]['score'])
    a.append(pre[0]['score'])
    a.append(pre[3]['score'])
    a.append(pre[1]['score'])
    a.append(pre[2]['score'])
    kluebert_pred_2.append(a)
    
kluebert_pred = np.array(kluebert_pred_2)

In [78]:
labels = {1: '01', 3: '03', 4: '04', 2: '02', 0: '00'}

In [79]:
pred_label = []

for kcbert, electra, kluebert in zip(kcbert_pred, kcelectra_pred, kluebert_pred):
    total = (kcbert + electra + kluebert)/3
    idx = np.argmax(total)
    pred_label.append(labels[idx])
    
pred_label[:10]

['01', '02', '02', '04', '01', '00', '00', '01', '04', '01']

In [82]:
test_file_path = '/aiffel/aiffel/aiffelthon/test.json'
with open(test_file_path, mode='rt', encoding='utf-8') as f:
    test_dataset = pd.read_json(f)
    
test_data = test_dataset.transpose()

submission = test_data.assign(CLASS=pred_label)
submission = submission.rename(columns={'CLASS':'class'})
submission.drop(['text'], axis=1, inplace=True)
submission = submission.transpose()

import json

submission_file_path = '/aiffel/aiffel/aiffelthon/submission/sub_voting.json'
result = submission.to_json(submission_file_path)

with open(submission_file_path) as f:
    parsed = json.load(f)

with open(submission_file_path, 'w') as f:
    json.dump(parsed, f, indent=4)